# Radiative Transfer for Roodeplaat on 2016-06-05

This notebook is essentially the libRadtran equivalent of the Matlab scripts that perform the radiative transfer calculations for the Sentinel 3 overpass on Sunday 2016-06-05.

In [1]:
import morticia.rad.librad as librad
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipyparallel import Client  # This is the client class for contacting the hub/controller
import datetime
import pytz
from scipy.optimize import curve_fit  # For fitting of Angstrom law to aerosol optical thickness measurements
# use latex for font rendering
mpl.rcParams['text.usetex'] = True  # Use TeX to format labels (takes a bit longer)
%matplotlib inline

In [49]:
# Create a blank case
S3 = librad.Case(casename='S3toTOAon20160605')

In [50]:
# Set revision
revision = '00A'
# Set up dates and times
overpass_datetime = datetime.datetime(2016, 6, 5, 7, 42, 31, tzinfo=pytz.utc)  # Overpass date and time down to second
overpass_datestr = overpass_datetime.strftime('%Y%m%d')
# Get the day of year
day_of_year = int(overpass_datetime.strftime('%j'))
S3.set_option('day_of_year', day_of_year)

results_folder = 'ResultsS3on' + overpass_datestr + 'Rev' + revision


# Choose basic atmospheric profile
atmos_profile = '../data/atmmod/afglmw.dat'
S3.set_option('atmosphere_file', atmos_profile)  # mid-latitude winter standard atmosphere

# Modify the ozone and water vapour column amounts
S3.set_option('mol_modify H2O', 10.5, 'MM')  # Total water vapour column in mm
S3.set_option('mol_modify O3', 266.1, 'DU')  # Total ozone column in Dobson units

# Change to the Thuillier spectrum and set wavelength range appropriately
solar_toa_file = '../data/solar_flux/Solar_irradiance_Thuillier_2002.txt'

# Pick the band model
S3.set_option()
# Choose start and stop wavelengths and minimum edge margin in nm
wv_minimum_range = [[385.0, -2.0], [955.0, 2.0]]
# Find the nearest wavelengths in the solar file
wv_range = librad.lookup_nearest_in_file('data/Solar_irradiance_Thuillier_2002.txt', wv_minimum_range)  # lookup in local copy
S3.set_option('source solar', solar_toa_file)
S3.set_option('wavelength', wv_range[0], wv_range[1])  # Wavelength must exist in the source solar file

# Set ground altitude
S3.set_option('altitude', 1.225)  # ground altitude in km above sea level

# Set up aerosol model
S3.set_option('aerosol_haze', 5)  # Shettle/Fenn urban aerosol
aot_wv = np.array([440, 500, 675, 870], dtype=np.float)
aot = np.array([0.703, 0.615, 0.362, 0.206])
# Define Angstrom law for wavelengths in micron - see uvspec manual aerosol_angstrom
def angstrom_law(wavelength, alpha, beta):
    return beta * wavelength ** (-alpha)
# Fit Angstrom law to data
popt, pcov = curve_fit(angstrom_law, aot_wv/1000.0, aot)  # Wavelengths must be in micron for Angstrom law purposes
alpha, beta = popt[0], popt[1]
S3.set_option('aerosol_angstrom', alpha, beta)

# Set up viewing and solar geometry. Note that these angles are taken from the S3 product and special
# care has to be taken when putting geometry information into libRadtran
OAA = 104.01066  # deg. Observation azimuth angle (presumably relative to north through east, satellite from dam)
OZA = 14.151356  # deg. Observation zenith angle (satellite zenith angle as seen from the dam)
SAA = 38.719933  # deg. Solar azimuth angle (presumably relative to north through east)
SZA = 59.316036  # deg. Solar zenith angle

S3.set_option('sza', SZA)  # deg. This one is straightforward
# Now when entering solar and observation zenith angles, it is necessary to provide the azimuth of light propagation
# rather than the azimuth of the view direction, which is 180 deg different
S3.set_option('phi0', 180.0 - SAA)  # solar radiation propagation azimuth from north through east
S3.set_option('phi', OAA)  # This is the azimuth of the satellite as seen from the target - also azimuth of light propgation
S3.set_option('umu', np.cos(np.deg2rad(OZA))) # For downward-looking (upward propagating), check that umu is positive


In [51]:
print S3

day_of_year 157
atmosphere_file ../data/atmmod/afglmw.dat
mol_modify H2O 10.5 MM
mol_modify O3 266.1 DU
source solar ../data/solar_flux/Solar_irradiance_Thuillier_2002.txt
wavelength 382.93 957.03
altitude 1.225
aerosol_haze 5
aerosol_angstrom 1.68488875619 0.181751188084
sza 59.316036
phi0 141.280067
phi 104.01066
umu 0.969653265954
